In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install streamlit


In [ ]:
!pip install annoy
!pip install pyngrok

In [ ]:
from pyngrok import ngrok


In [ ]:
import streamlit as st
import pickle
import numpy as np
import cv2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
from annoy import AnnoyIndex
import matplotlib.pyplot as plt


In [ ]:
feature_list = np.array(pickle.load(open('/content/drive/My Drive/FatimaQurban_ML&DL_BytewiseFellowship2024/FatimaQurban_ML&DL_FashioRecommendationSystem/new_embeddings.pkl','rb')))
filenames = pickle.load(open('/content/drive/My Drive/FatimaQurban_ML&DL_BytewiseFellowship2024/FatimaQurban_ML&DL_FashioRecommendationSystem/new_filenames.pkl','rb'))

In [ ]:
# Load ResNet model for image feature extraction
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import GlobalMaxPooling2D
from tensorflow.keras.models import Model

base_model = ResNet50(weights='imagenet', include_top=False)
model = Model(inputs=base_model.input, outputs=GlobalMaxPooling2D()(base_model.output))

In [ ]:
st.title("Fashion Recommendation System")
st.write("Upload an image to get similar fashion item recommendations.")

# File uploader for user input
uploaded_file = st.file_uploader("Choose an image...", type="jpg")

if uploaded_file is not None:
    st.image(uploaded_file, caption='Uploaded Image', use_column_width=True)
    st.write("Processing...")

    # Load and preprocess the uploaded image
    img = image.load_img(uploaded_file, target_size=(224, 224))
    img_array = image.img_to_array(img)
    expanded_img_array = np.expand_dims(img_array, axis=0)
    preprocessed_img = preprocess_input(expanded_img_array)

    # Get feature embeddings
    result = model.predict(preprocessed_img).flatten()
    normalized_result = result / np.linalg.norm(result)

    # Initialize Annoy index
    annoy_index = AnnoyIndex(feature_list.shape[1], metric='euclidean')
    annoy_index.build(10)

    # Find similar images using Annoy
    indices = annoy_index.get_nns_by_vector(normalized_result, 6)

    st.write("Recommendations:")

    for i in indices[1:]:  # Skip the first index, as it's the query image
        img_path = filenames[i]
        temp_img = cv2.imread(img_path)
        resized_img = cv2.resize(temp_img, (512, 512))
        rgb_img = cv2.cvtColor(resized_img, cv2.COLOR_BGR2RGB)
        st.image(rgb_img, width=300)

2024-09-10 20:17:18.403 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-10 20:17:18.407 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-10 20:17:18.412 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-10 20:17:18.415 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-10 20:17:18.418 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-10 20:17:18.421 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-10 20:17:18.423 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-09-10 20:17:18.426 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [ ]:
!streamlit run app.py &>/content/logs.txt &


In [ ]:
!ngrok authtoken


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
public_url = ngrok.connect(8501, "http")
print(public_url)


NgrokTunnel: "https://e5e4-34-75-78-29.ngrok-free.app" -> "http://localhost:8501"
